## **MNIST - CNN**

#### **Cargar datos y procesar**

In [2]:
from sklearn.datasets import fetch_openml
from tensorflow import keras
import numpy as np
import tensorflow as tf


In [11]:
mnist = fetch_openml('mnist_784', version=1, as_frame = False)

In [12]:
X, y = mnist["data"], mnist["target"]

In [13]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [14]:
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

### **Version 1**

In [18]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    
    keras.layers.Dense(10, activation='softmax')  # 10 clases para las 10 cifras de 0-9
])

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 [==============================] - 89s 52ms/step - loss: 0.1542 - accuracy: 0.9534 - val_loss: 0.0478 - val_accuracy: 0.9865
Epoch 2/5
1688/1688 [==============================] - 90s 53ms/step - loss: 0.0483 - accuracy: 0.9851 - val_loss: 0.0463 - val_accuracy: 0.9867
Epoch 3/5
1688/1688 [==============================] - 94s 56ms/step - loss: 0.0340 - accuracy: 0.9895 - val_loss: 0.0416 - val_accuracy: 0.9890
Epoch 4/5
1688/1688 [==============================] - 80s 48ms/step - loss: 0.0244 - accuracy: 0.9924 - val_loss: 0.0399 - val_accuracy: 0.9890
Epoch 5/5
1688/1688 [==============================] - 90s 53ms/step - loss: 0.0192 - accuracy: 0.9939 - val_loss: 0.0409 - val_accuracy: 0.9902


In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc

313/313 [==============================] - 2s 7ms/step - loss: 0.0270 - accuracy: 0.9914


0.9914000034332275

### **Version 2**

In [23]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),  
    
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.4),  
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),  
    
    keras.layers.Dense(10, activation='softmax')  # 10 clases para las 10 cifras de 0-9
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 417s 246ms/step - loss: 0.4067 - accuracy: 0.8744 - val_loss: 0.0641 - val_accuracy: 0.9805
Epoch 2/10
1688/1688 [==============================] - 258s 153ms/step - loss: 0.1436 - accuracy: 0.9562 - val_loss: 0.0606 - val_accuracy: 0.9823
Epoch 3/10
1688/1688 [==============================] - 312s 185ms/step - loss: 0.1140 - accuracy: 0.9666 - val_loss: 0.0410 - val_accuracy: 0.9877
Epoch 4/10
1688/1688 [==============================] - 228s 135ms/step - loss: 0.0954 - accuracy: 0.9716 - val_loss: 0.0425 - val_accuracy: 0.9872
Epoch 5/10
1688/1688 [==============================] - 180s 107ms/step - loss: 0.0874 - accuracy: 0.9746 - val_loss: 0.0364 - val_accuracy: 0.9877
Epoch 6/10
1688/1688 [==============================] - 329s 195ms/step - loss: 0.0742 - accuracy: 0.9775 - val_loss: 0.0406 - val_accuracy: 0.9875
Epoch 7/10
1688/1688 [==============================] - 456s 270ms/step - loss: 0.0728 - accuracy: 0.9781 - val_

In [25]:
test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc

313/313 [==============================] - 3s 10ms/step - loss: 0.0423 - accuracy: 0.9867


0.9866999983787537

### **Version 3**

In [21]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
])

model = keras.models.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),  

    data_augmentation,  

    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.GlobalAveragePooling2D(),
    
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(10, activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


In [22]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 408s 238ms/step - loss: 0.3188 - accuracy: 0.9011 - val_loss: 0.0906 - val_accuracy: 0.9735
Epoch 2/10
1688/1688 [==============================] - 379s 224ms/step - loss: 0.1311 - accuracy: 0.9599 - val_loss: 0.0899 - val_accuracy: 0.9752
Epoch 3/10
1688/1688 [==============================] - 398s 235ms/step - loss: 0.1020 - accuracy: 0.9682 - val_loss: 0.1739 - val_accuracy: 0.9453
Epoch 4/10
1688/1688 [==============================] - 416s 247ms/step - loss: 0.0846 - accuracy: 0.9736 - val_loss: 0.0631 - val_accuracy: 0.9822
Epoch 5/10
1688/1688 [==============================] - 787s 466ms/step - loss: 0.0771 - accuracy: 0.9761 - val_loss: 0.0734 - val_accuracy: 0.9775
Epoch 6/10
1688/1688 [==============================] - 584s 346ms/step - loss: 0.0696 - accuracy: 0.9778 - val_loss: 0.0529 - val_accuracy: 0.9843
Epoch 7/10
1688/1688 [==============================] - 738s 437ms/step - loss: 0.0671 - accuracy: 0.9795 - val_

In [23]:
test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc

313/313 [==============================] - 2s 5ms/step - loss: 0.0599 - accuracy: 0.9824


0.9824000000953674

## **Fashon MNIST-CNN**

#### **Cargar datos y procesar**

In [24]:
import pandas as pd 
import numpy as np 

In [25]:
train_df = pd.read_csv('fashion-mnist_train.csv')
test_df = pd.read_csv('fashion-mnist_test.csv')

In [26]:
y_train = train_df.pop("label")
y_test = test_df.pop("label")
X_train = np.array(train_df)
X_test= np.array(test_df)

In [27]:
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

### **Verison 1**

In [28]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    
    keras.layers.Dense(10, activation='softmax')  # 10 clases para las 10 cifras de 0-9
])

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 [==============================] - 83s 49ms/step - loss: 0.4956 - accuracy: 0.8212 - val_loss: 0.3713 - val_accuracy: 0.8617
Epoch 2/5
1688/1688 [==============================] - 86s 51ms/step - loss: 0.3272 - accuracy: 0.8820 - val_loss: 0.3123 - val_accuracy: 0.8893
Epoch 3/5
1688/1688 [==============================] - 90s 53ms/step - loss: 0.2791 - accuracy: 0.8981 - val_loss: 0.2829 - val_accuracy: 0.8997
Epoch 4/5
1688/1688 [==============================] - 82s 49ms/step - loss: 0.2472 - accuracy: 0.9089 - val_loss: 0.2664 - val_accuracy: 0.9008
Epoch 5/5
1688/1688 [==============================] - 90s 53ms/step - loss: 0.2225 - accuracy: 0.9186 - val_loss: 0.2545 - val_accuracy: 0.9075


In [30]:
test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc

313/313 [==============================] - 1s 2ms/step - loss: 0.2472 - accuracy: 0.9099


0.9099000096321106

### **Verison 2**

In [31]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),  
    
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.4),  
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),  
    
    keras.layers.Dense(10, activation='softmax')  # 10 clases para las 10 cifras de 0-9
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 393s 232ms/step - loss: 0.7760 - accuracy: 0.7309 - val_loss: 0.4503 - val_accuracy: 0.8367
Epoch 2/10
1688/1688 [==============================] - 427s 253ms/step - loss: 0.5031 - accuracy: 0.8208 - val_loss: 0.3852 - val_accuracy: 0.8585
Epoch 3/10
1688/1688 [==============================] - 3578s 2s/step - loss: 0.4439 - accuracy: 0.8427 - val_loss: 0.3391 - val_accuracy: 0.8742
Epoch 4/10
1688/1688 [==============================] - 447s 265ms/step - loss: 0.4089 - accuracy: 0.8561 - val_loss: 0.3225 - val_accuracy: 0.8847
Epoch 5/10
1688/1688 [==============================] - 437s 259ms/step - loss: 0.3849 - accuracy: 0.8652 - val_loss: 0.3206 - val_accuracy: 0.8783
Epoch 6/10
1688/1688 [==============================] - 587s 348ms/step - loss: 0.3672 - accuracy: 0.8717 - val_loss: 0.3395 - val_accuracy: 0.8715
Epoch 7/10
1688/1688 [==============================] - 355s 210ms/step - loss: 0.3565 - accuracy: 0.8745 - val_lo

In [33]:
test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc

313/313 [==============================] - 1s 4ms/step - loss: 0.2687 - accuracy: 0.9005


0.9004999995231628

### **Verison 3**

In [34]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
])

model = keras.models.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),  

    data_augmentation,  

    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.GlobalAveragePooling2D(),
    
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(10, activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


In [36]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 403s 238ms/step - loss: 0.6402 - accuracy: 0.7698 - val_loss: 1.9807 - val_accuracy: 0.6137
Epoch 2/10
1688/1688 [==============================] - 391s 231ms/step - loss: 0.5003 - accuracy: 0.8189 - val_loss: 2.1987 - val_accuracy: 0.5717
Epoch 3/10
1688/1688 [==============================] - 396s 234ms/step - loss: 0.4499 - accuracy: 0.8385 - val_loss: 2.4216 - val_accuracy: 0.5927
Epoch 4/10
1688/1688 [==============================] - 404s 239ms/step - loss: 0.4154 - accuracy: 0.8499 - val_loss: 1.9259 - val_accuracy: 0.6312
Epoch 5/10
1688/1688 [==============================] - 394s 233ms/step - loss: 0.3935 - accuracy: 0.8575 - val_loss: 1.9266 - val_accuracy: 0.6272
Epoch 6/10
1688/1688 [==============================] - 391s 232ms/step - loss: 0.3792 - accuracy: 0.8637 - val_loss: 1.4924 - val_accuracy: 0.6840
Epoch 7/10
1688/1688 [==============================] - 399s 237ms/step - loss: 0.3594 - accuracy: 0.8710 - val_

In [37]:
test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc

313/313 [==============================] - 2s 7ms/step - loss: 1.6465 - accuracy: 0.6853


0.6852999925613403